In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, r2_score, roc_auc_score, log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import KFold
from sklearn.naive_bayes import BernoulliNB

In [2]:
glass = pd.read_csv(r'C:\Users\Administrator.DAI-PC2\Desktop\jupiter_demo\ML\Cases-20240426T111123Z-001\Cases\Glass Identification\Glass.csv')
le = LabelEncoder()
y = le.fit_transform(glass['Type'])
X = glass.drop(['Type'], axis=1)
print(le.classes_)

['building_windows_float_processed' 'building_windows_non_float_processed'
 'containers' 'headlamps' 'tableware' 'vehicle_windows_float_processed']


In [21]:
glass['Type'].value_counts()

Type
building_windows_non_float_processed    76
building_windows_float_processed        70
headlamps                               29
vehicle_windows_float_processed         17
containers                              13
tableware                                9
Name: count, dtype: int64

In [12]:
lr = LogisticRegression(solver='saga', random_state=24, multi_class='multinomial')

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=24,stratify=y)

In [7]:
lr.fit(X_train,y_train)

C:\Users\Administrator.DAI-PC2\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [8]:
y_pred = lr.predict(X_test)
y_pred

array([0, 3, 1, 0, 0, 0, 4, 1, 0, 0, 3, 1, 1, 1, 0, 3, 3, 1, 1, 1, 1, 3,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 3, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       3, 3, 0, 1, 1, 2, 0, 1, 3, 0, 1, 0, 3, 0, 0, 1, 0, 0, 2, 0, 1])

In [9]:
y_pred_prob = lr.predict_proba(X_test)

In [11]:
log_loss(y_test, y_pred_prob)

1.013441901293348

In [15]:
kfold = StratifiedKFold(n_splits =5, shuffle=True, random_state=24)
params = {'penalty':['elasticnet', 'l1','l2',None], 'C':np.linspace(0.001,10,5), 'l1_ratio':np.linspace(0.001,1,4), 'multi_class':['ovr','multinomial']}

In [19]:

import warnings
warnings.filterwarnings('ignore')

gcv = GridSearchCV(lr, param_grid=params, cv=kfold, scoring='neg_log_loss')
gcv.fit(X,y)

pd_cv = pd.DataFrame(gcv.cv_results_)
print(gcv.best_params_)
print(gcv.best_score_)

{'C': 0.001, 'l1_ratio': 0.001, 'multi_class': 'multinomial', 'penalty': None}
-1.098224924028305


In [22]:
pd_cv

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_l1_ratio,param_multi_class,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015796,0.004045,0.002788,0.000745,0.001,0.001,ovr,elasticnet,"{'C': 0.001, 'l1_ratio': 0.001, 'multi_class':...",-1.442489,-1.488411,-1.487411,-1.485755,-1.485321,-1.477877,0.017729,146
1,0.017986,0.006896,0.003125,0.006251,0.001,0.001,ovr,l1,"{'C': 0.001, 'l1_ratio': 0.001, 'multi_class':...",-1.526491,-1.550434,-1.549995,-1.551810,-1.544147,-1.544575,0.009416,150
2,0.012726,0.007087,0.001802,0.000979,0.001,0.001,ovr,l2,"{'C': 0.001, 'l1_ratio': 0.001, 'multi_class':...",-1.440801,-1.486954,-1.486058,-1.484056,-1.484116,-1.476397,0.017833,142
3,0.015736,0.001782,0.003126,0.006252,0.001,0.001,ovr,None,"{'C': 0.001, 'l1_ratio': 0.001, 'multi_class':...",-1.052970,-1.148950,-1.173260,-1.090066,-1.169541,-1.126957,0.047489,69
4,0.006262,0.007670,0.003125,0.006251,0.001,0.001,multinomial,elasticnet,"{'C': 0.001, 'l1_ratio': 0.001, 'multi_class':...",-1.432389,-1.480529,-1.479749,-1.476451,-1.477680,-1.469360,0.018542,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.021989,0.006092,0.000000,0.000000,10.0,1.0,ovr,None,"{'C': 10.0, 'l1_ratio': 1.0, 'multi_class': 'o...",-1.052970,-1.148950,-1.173260,-1.090066,-1.169541,-1.126957,0.047489,69
156,0.014825,0.001631,0.004037,0.006057,10.0,1.0,multinomial,elasticnet,"{'C': 10.0, 'l1_ratio': 1.0, 'multi_class': 'm...",-1.011763,-1.126574,-1.142098,-1.055356,-1.161765,-1.099511,0.056694,46
157,0.014146,0.003820,0.003524,0.006099,10.0,1.0,multinomial,l1,"{'C': 10.0, 'l1_ratio': 1.0, 'multi_class': 'm...",-1.011763,-1.126574,-1.142098,-1.055356,-1.161765,-1.099511,0.056694,46
158,0.011876,0.006017,0.003728,0.006065,10.0,1.0,multinomial,l2,"{'C': 10.0, 'l1_ratio': 1.0, 'multi_class': 'm...",-1.010185,-1.125566,-1.141311,-1.054223,-1.161108,-1.098479,0.057000,21
